# Проект: классификация

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import plotly
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

from  sklearn.ensemble import IsolationForest
import warnings
warnings.filterwarnings('ignore')
from sklearn.preprocessing  import LabelEncoder
from sklearn import linear_model 
from sklearn import tree 
from sklearn import ensemble 
from sklearn import metrics 
from sklearn import preprocessing 
from sklearn.model_selection import train_test_split 
from sklearn.feature_selection import SelectKBest, f_classif

## Часть 1. Знакомство с данными, обработка пропусков и выбросов

Данные о клиентах банка:

- `age` (возраст);
- `job` (сфера занятости);
- `marital` (семейное положение);
- `education` (уровень образования);
- `default` (имеется ли просроченный кредит);
- `housing` (имеется ли кредит на жильё);
- `loan` (имеется ли кредит на личные нужды);
- `balance` (баланс).

Данные, связанные с последним контактом в контексте текущей маркетинговой кампании:

- `contact` (тип контакта с клиентом);
- `month` (месяц, в котором был последний контакт);
- `da`y (день, в который был последний контакт);
- `duration` (продолжительность контакта в секундах).

Прочие признаки:

- `campaign` (количество контактов с этим клиентом в течение текущей кампании);
- `pdays` (количество пропущенных дней с момента последней маркетинговой кампании до контакта в текущей кампании);
- `previous` (количество контактов до текущей кампании)
- `poutcome` (результат прошлой маркетинговой кампании).

И, разумеется, наша целевая переменная `deposit` , которая определяет, согласится ли клиент открыть депозит в банке. Именно её мы будем пытаться предсказать в данном кейсе.

### Задание 1

In [2]:
df = pd.read_csv('data/bank_fin.csv', sep = ';')

In [94]:
# исследуйте данные на предмет пропусков. Где есть пропущенные значения? Сколько их?
display(df.info())

hotels_isnull = df.isnull().sum()
print(
    f'\nСтолбцы с явными пропусками данных:\n{hotels_isnull[hotels_isnull > 0]}'
)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11162 entries, 0 to 11161
Data columns (total 17 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   age        11162 non-null  int64  
 1   job        11162 non-null  object 
 2   marital    11162 non-null  object 
 3   education  11162 non-null  object 
 4   default    11162 non-null  object 
 5   balance    11162 non-null  float64
 6   housing    11162 non-null  object 
 7   loan       11162 non-null  object 
 8   contact    11162 non-null  object 
 9   day        11162 non-null  int64  
 10  month      11162 non-null  object 
 11  duration   11162 non-null  int64  
 12  campaign   11162 non-null  int64  
 13  pdays      11162 non-null  int64  
 14  previous   11162 non-null  int64  
 15  poutcome   11162 non-null  object 
 16  deposit    11162 non-null  object 
dtypes: float64(1), int64(6), object(10)
memory usage: 1.4+ MB


None


Столбцы с явными пропусками данных:
Series([], dtype: int64)


### Задание 2

In [95]:
# есть ли в признаке job пропущенные значения? Возможно, они обозначены каким-то специальным словом?
display(df["job"].value_counts())
df[df["job"] == "unknown"].head(10)

management       2636
blue-collar      1944
technician       1823
admin.           1334
services          923
retired           778
self-employed     405
student           360
unemployed        357
entrepreneur      328
housemaid         274
Name: job, dtype: int64

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,deposit


### Задание 3

In [96]:
# преобразуйте признак balance таким образом, чтобы он корректно считывался, как вещественное число (float)
df["balance"] = df["balance"].apply(lambda x: x if x is np.NaN else float("".join(x.split()[:-1]).replace(",", ".")))
round(df["balance"].mean(),3)

AttributeError: 'float' object has no attribute 'split'

### Задание 4

In [ ]:
# обработайте пропуски в признаки balance , заменив их на медианные значения по данному признаку
df.loc[(df['balance'].isnull()==True),'balance']=df['balance'].median()
round(df["balance"].mean(),3)

1526.936

### Задание 5

In [ ]:
# обработайте пропуски в категориальных признаках: job и education, заменив их на модальные значения
df.loc[(df['job']== "unknown"),'job']=df['job'].mode()[0]
df.loc[(df['education']== "unknown"),'education']=df['education'].mode()[0]



In [ ]:
job_mask = df['job']== df["job"].mode()[0]
education_mask = df['education']== df["education"].mode()[0]
round(df[job_mask & education_mask]["balance"].mean(),3)


1598.883

### Задание 6

In [ ]:
# удалите все выбросы для признака balance
def outlires_iqr(data,feature):
    x =data[feature]
    quartel_1,quartel_3 = x.quantile(0.25),x.quantile(0.75)
    iqr = quartel_3 - quartel_1
    lower_bound = quartel_1 - (iqr * 1.5)
    upper_bound = quartel_3 + (iqr * 1.5)
    outlires = data[(x < lower_bound)|(x > upper_bound)]
    cleaned = data[(x >= lower_bound)&(x <= upper_bound)]
    return outlires,cleaned,lower_bound,upper_bound

outliers, cleaned,lower_bound,upper_bound = outlires_iqr(df,'balance')
display(outliers.shape[0])
display(lower_bound)
display(upper_bound)

1057

-2241.0

4063.0

## Часть 2:  Разведывательный анализ

### Задание 1

In [ ]:
# изучите соотношение классов в ваших данных на предмет несбалансированности, проиллюстрируйте результат
fig = px.histogram(
    data_frame=cleaned,
    x='deposit',
    color="deposit",
    barmode='group',
    histfunc='count',
    title='Соотношение соглошившихся и отказавшихся от депозита',
    text_auto=True
)
fig.show()



### Задания 2 и 3

In [ ]:
#рассчитайте описательные статистики для количественных переменных, проинтерпретируйте результат
display(cleaned.describe())
fig_2 = px.bar(cleaned, x="age", y="balance", color="deposit", title="Распределение баланса по возрасту и наличию депозита")
fig_2.show()

,age,balance,day,duration,campaign,pdays,previous
count,10105.000000,10105.000000,10105.000000,10105.000000,10105.000000,10105.000000,10105.000000
mean,40.895497,807.653538,15.590302,368.742603,2.517170,51.319644,0.816230
std,11.734931,994.151966,8.441510,346.651524,2.707159,109.644179,2.243795
min,18.000000,-2049.000000,1.000000,2.000000,1.000000,-1.000000,0.000000
25%,32.000000,95.000000,8.000000,137.000000,1.000000,-1.000000,0.000000
50%,38.000000,445.000000,15.000000,252.000000,2.000000,-1.000000,0.000000
75%,48.000000,1227.000000,22.000000,490.000000,3.000000,2.000000,1.000000
max,95.000000,4063.000000,31.000000,3881.000000,43.000000,854.000000,58.000000


### Задания 4 и 5

In [ ]:
#рассчитайте описательные статистики для категориальных переменных, проинтерпретируйте результат
cleaned.describe(include='object')
#постройте визуализации, иллюстрирующие результаты

,job,marital,education,default,housing,loan,contact,month,poutcome,deposit
count,10105,10105,10105,10105,10105,10105,10105,10105,10105,10105
unique,11,3,3,2,2,2,3,12,4,2
top,management,married,secondary,no,no,no,cellular,may,unknown,no
freq,2315,5715,5517,9939,5243,8712,7283,2617,7570,5424


In [ ]:
fig = make_subplots(rows=4, cols=2,subplot_titles=["job", "marital", "education", "default","housing","loan","contact","month"])

fig.add_trace(go.Histogram(x = cleaned["job"],histfunc="count"), row=1, col=1)
fig.add_trace(go.Histogram(x = cleaned[cleaned["deposit"] == "no"]["marital"],histfunc="count",name="no_dep"), row=1, col=2)
fig.add_trace(go.Histogram(x = cleaned[cleaned["deposit"] == "yes"]["marital"],histfunc="count"), row=1, col=2)

fig.add_trace(go.Histogram(x = cleaned["education"],histfunc="count"), row=2, col=1)
fig.add_trace(go.Histogram(x = cleaned[cleaned["deposit"] == "no"]["education"],histfunc="count"), row=2, col=2)
fig.add_trace(go.Histogram(x = cleaned[cleaned["deposit"] == "yes"]["education"],histfunc="count"), row=2, col=2)

fig.add_trace(go.Histogram(x = cleaned["housing"],histfunc="count"), row=3, col=1)
fig.add_trace(go.Histogram(x = cleaned["education"],histfunc="count"), row=3, col=2)
fig.update_layout(width=1200, height=1800)
fig.update_layout(barmode='stack')
#fig.update_layout(legend_title_font_color=["#FF0000", "#00FF00", "#0000FF","#FF0000", "#00FF00", "#0000FF","#0000FF"])
fig.show()



In [109]:

# Выберите 8 категориальных признаков для визуализации
categorical_features = ["job", "marital", "education", "default","housing","loan","contact","month"]

# Создайте подзаголовки для графиков
titles = [f'Distribution of {feature}' for feature in categorical_features]

# Создайте сетку с 2 рядами и 4 столбцами для графиков
fig = make_subplots(rows=3, cols=4, subplot_titles=titles, horizontal_spacing=0.1)

# Постройте графики для каждого категориального признака
for i, feature in enumerate(categorical_features):
    row = i // 4 + 1  # Определите строку для текущего графика
    col = i % 4 + 1   # Определите столбец для текущего графика
    
    fig.add_trace(px.histogram(data_frame=cleaned, x=feature,text_auto=True,color="deposit").data[0], row=row, col=col)
    fig.add_trace(px.histogram(data_frame=cleaned, x=feature,text_auto=True,color="deposit").data[1], row=row, col=col)
fig.add_trace(px.histogram(data_frame=cleaned,x="poutcome",color="deposit",text_auto=True).data[0],row=3,col=1)
fig.add_trace(px.histogram(data_frame=cleaned,x="poutcome",color="deposit",text_auto=True).data[1],row=3,col=1)
# Обновите макет графиков
fig.update_layout(showlegend=True, title_text='Categorical Feature Distributions')
fig.update_layout(width=1200, height=900)
fig.update_layout(barmode='stack')
# Отобразите графики
fig.show()

### Задание 6

In [100]:
# Узнайте, для какого статуса предыдущей маркетинговой кампании успех в текущей превалирует над количеством неудач
fig_6 = px.histogram(data_frame=cleaned,x="poutcome",color="deposit",text_auto=True)
fig_6.show()

### Задание 7

In [ ]:
# узнайте, в каком месяце чаще всего отказывались от предложения открыть депозит
# ваш код

### Задание 8

In [ ]:
# создайте возрастные группы и определите, в каких группах более склонны открывать депозит, чем отказываться от предложения

### Задания 9 и 10

In [ ]:
# постройте визуализации для открывших и неоткрывших депозит в зависимости от семейного статуса

In [ ]:
# постройте визуализации для открывших и неоткрывших депозит в зависимости от образования

In [ ]:
# постройте визуализации для открывших и неоткрывших депозит в зависимости от вида профессиональной занятости

### Задание 11

In [ ]:
# постройте сводную таблицу, чтобы определить люди с каким образованием и семейным статусом наиболее многочисленны
#(если рассматривать тех, кто открыл депозит)

## Часть 3: преобразование данных

### Задание 1

In [ ]:
# преобразуйте уровни образования

### Задания 2 и 3

In [ ]:
# преобразуйте бинарные переменные в представление из нулей и единиц

### Задание 4

In [ ]:
# создайте дамми-переменные

### Задания 5 и 6

In [ ]:
# постройте корреляционную матрицу и оцените данные на предмет наличия мультиколлинеарности

### Задания 7 и 8

In [ ]:
X = df.drop(['deposit'], axis=1)
y = df['deposit']
 
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, random_state = 42, test_size = 0.33)

In [ ]:
# рассчитайте необходимые показатели

### Задание 9

In [ ]:
# с помощью SelectKBest отберите 15 наиболее подходящих признаков

### Задание 10

In [ ]:
# нормализуйте данные с помощью minmaxsxaler

# Часть 4: Решение задачи классификации: логистическая регрессия и решающие деревья

### Задание 1

In [ ]:
# обучите логистическую регрессию и рассчитайте метрики качества

### Задания 2,3,4

In [ ]:
# обучите решающие деревья, настройте максимальную глубину

### Задание 5

In [ ]:
# подберите оптимальные параметры с помощью gridsearch

# Часть 5: Решение задачи классификации: ансамбли моделей и построение прогноза

### Задание 1

In [ ]:
# обучите на ваших данных случайный лес

### Задания 2 и 3

In [ ]:
# используйте для классификации градиентный бустинг и сравните качество со случайным лесом

### Задание 4

In [ ]:
# объедините уже известные вам алгоритмы с помощью стекинга 

### Задание 5

In [ ]:
# оцените, какие признаки демонстрируют наибольшую  важность в модели градиентного бустинга

### Задания 6,7,8

In [ ]:
# реализуйте оптимизацию гиперпараметров с помощью Optuna